# 🎯 Week 22, Day 1: Recommender Systems

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gouthamgo/Learn-AI/blob/main/Phase-4-Advanced-AI/Week-22-Job-Critical-Skills/Day-1-Recommender-Systems.ipynb)

## 🚀 Why This Matters

Recommender systems are **EVERYWHERE** in tech:
- 📺 **Netflix**: 80% of watched content comes from recommendations
- 🛒 **Amazon**: 35% of revenue from recommendations
- 🎵 **Spotify**: Discover Weekly has 40M+ active users
- 📱 **TikTok**: Entire platform built on recommendation algorithm

**Job Market Reality:**
- REA Group specifically lists "Recommender systems (Collaborative Filtering, Content-Based, Hybrid)" ✅
- E-commerce companies (Woolworths, Amazon) heavily use RecSys
- Social media, streaming, ad tech - all need RecSys engineers

## 📋 What You'll Learn Today

1. **Collaborative Filtering** - Learn from user behavior
   - User-User similarity
   - Item-Item similarity
2. **Matrix Factorization** - The Netflix Prize winner!
   - SVD (Singular Value Decomposition)
   - ALS (Alternating Least Squares)
3. **Content-Based Filtering** - Recommend similar items
4. **Hybrid Systems** - Combine the best of both worlds
5. **Deep Learning RecSys** - Neural Collaborative Filtering
6. **Learning to Rank** - Optimize for relevance
7. **🏆 Project: E-Commerce Recommendation Engine**

---

## Part 1: Understanding Recommender Systems

### Types of Recommender Systems

1. **Collaborative Filtering**: "Users who liked this also liked..."
   - User-based: Find similar users, recommend what they liked
   - Item-based: Find similar items, recommend those

2. **Content-Based**: "Since you liked X with features Y, try Z with similar features"
   - Uses item features/metadata
   - Independent of other users

3. **Hybrid**: Combine collaborative + content-based
   - Solves cold start problem
   - Better accuracy

4. **Deep Learning**: Neural networks for recommendations
   - Can learn complex patterns
   - Used by YouTube, TikTok

### Key Metrics

- **Precision@K**: Of K recommendations, how many are relevant?
- **Recall@K**: Of all relevant items, how many are in top K?
- **NDCG**: Normalized Discounted Cumulative Gain (considers ranking)
- **MAE/RMSE**: For rating prediction

---

In [ ]:
# Install required libraries
!pip install surprise scikit-learn pandas numpy matplotlib seaborn -q
!pip install implicit -q  # For ALS

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
import warnings
warnings.filterwarnings('ignore')

print("✅ All libraries installed!")

## Part 2: Collaborative Filtering

### 2.1 User-Based Collaborative Filtering

**Idea**: Find users similar to you, recommend what they liked.

**Example**:
- You liked: "Inception", "Interstellar", "The Matrix"
- User B (80% similar) also liked those + "Blade Runner"
- Recommendation: "Blade Runner" ✅

In [ ]:
# Create sample user-item rating matrix
ratings_data = {
    'user_id': [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5],
    'item_id': [101, 102, 103, 101, 102, 104, 102, 103, 104, 101, 103, 105, 102, 104, 105],
    'rating': [5, 4, 3, 5, 5, 4, 4, 5, 4, 5, 4, 3, 5, 5, 4]
}

ratings_df = pd.DataFrame(ratings_data)

# Create user-item matrix
user_item_matrix = ratings_df.pivot_table(
    index='user_id',
    columns='item_id',
    values='rating',
    fill_value=0
)

print("User-Item Rating Matrix:")
print(user_item_matrix)
print(f"\nShape: {user_item_matrix.shape} (users x items)")

In [ ]:
# Calculate user-user similarity
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(
    user_similarity,
    index=user_item_matrix.index,
    columns=user_item_matrix.index
)

print("User-User Similarity Matrix:")
print(user_similarity_df.round(3))

# Visualize
plt.figure(figsize=(8, 6))
sns.heatmap(user_similarity_df, annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title('User-User Similarity (Cosine)')
plt.tight_layout()
plt.show()

In [ ]:
# User-based recommendation function
def recommend_user_based(user_id, user_item_matrix, user_similarity_df, top_n=3):
    """
    Recommend items based on similar users' preferences.
    """
    # Get similarity scores for this user
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:]  # Exclude self
    
    # Get items this user hasn't rated
    user_ratings = user_item_matrix.loc[user_id]
    unrated_items = user_ratings[user_ratings == 0].index
    
    # Calculate predicted ratings
    predictions = {}
    for item in unrated_items:
        # Weighted average of similar users' ratings
        numerator = 0
        denominator = 0
        
        for similar_user in similar_users.index:
            rating = user_item_matrix.loc[similar_user, item]
            similarity = similar_users[similar_user]
            
            if rating > 0:  # User rated this item
                numerator += rating * similarity
                denominator += similarity
        
        if denominator > 0:
            predictions[item] = numerator / denominator
    
    # Sort and return top N
    sorted_predictions = sorted(predictions.items(), key=lambda x: x[1], reverse=True)
    return sorted_predictions[:top_n]

# Test recommendations
user_id = 1
recommendations = recommend_user_based(user_id, user_item_matrix, user_similarity_df)

print(f"\n🎯 Top Recommendations for User {user_id}:")
for item, score in recommendations:
    print(f"  Item {item}: Predicted Rating = {score:.2f}")

### 2.2 Item-Based Collaborative Filtering

**Idea**: "You liked item A, here's item B which is similar to A"

**Advantages over user-based**:
- Items change less frequently than user preferences
- More stable recommendations
- Easier to explain: "Because you bought X, you might like Y"

**Used by**: Amazon, YouTube

In [ ]:
# Calculate item-item similarity
item_similarity = cosine_similarity(user_item_matrix.T)  # Transpose!
item_similarity_df = pd.DataFrame(
    item_similarity,
    index=user_item_matrix.columns,
    columns=user_item_matrix.columns
)

print("Item-Item Similarity Matrix:")
print(item_similarity_df.round(3))

# Visualize
plt.figure(figsize=(8, 6))
sns.heatmap(item_similarity_df, annot=True, fmt='.2f', cmap='viridis')
plt.title('Item-Item Similarity (Cosine)')
plt.tight_layout()
plt.show()

In [ ]:
# Item-based recommendation function
def recommend_item_based(user_id, user_item_matrix, item_similarity_df, top_n=3):
    """
    Recommend items similar to what user already liked.
    """
    # Get items this user rated
    user_ratings = user_item_matrix.loc[user_id]
    rated_items = user_ratings[user_ratings > 0]
    unrated_items = user_ratings[user_ratings == 0].index
    
    # Calculate predicted ratings
    predictions = {}
    for item in unrated_items:
        # Weighted average based on similar items
        numerator = 0
        denominator = 0
        
        for rated_item, rating in rated_items.items():
            similarity = item_similarity_df.loc[item, rated_item]
            numerator += rating * similarity
            denominator += similarity
        
        if denominator > 0:
            predictions[item] = numerator / denominator
    
    # Sort and return top N
    sorted_predictions = sorted(predictions.items(), key=lambda x: x[1], reverse=True)
    return sorted_predictions[:top_n]

# Test recommendations
recommendations = recommend_item_based(user_id, user_item_matrix, item_similarity_df)

print(f"\n🎯 Item-Based Recommendations for User {user_id}:")
for item, score in recommendations:
    print(f"  Item {item}: Predicted Rating = {score:.2f}")

## Part 3: Matrix Factorization (The Netflix Prize Winner!)

### Why Matrix Factorization?

**Problem with CF**: User-item matrix is HUGE and SPARSE
- Netflix: 500K users × 20K movies = 10 billion entries
- Most users rate <1% of items
- 99%+ sparsity!

**Solution**: Decompose into smaller matrices
- User-Item Matrix (m × n) → User-Factor (m × k) × Factor-Item (k × n)
- k = 10-100 latent factors (vs thousands of items)

### SVD (Singular Value Decomposition)

**Math**: R ≈ U × Σ × V^T
- U: User embeddings in latent space
- Σ: Importance of each factor
- V: Item embeddings in latent space

**Surprise library**: Production-ready SVD implementation

In [ ]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate, train_test_split
from surprise import accuracy

# Create dataset in Surprise format
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_df[['user_id', 'item_id', 'rating']], reader)

# Split train/test
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

# Train SVD model
svd_model = SVD(
    n_factors=10,      # Number of latent factors (k)
    n_epochs=20,       # Training iterations
    lr_all=0.005,      # Learning rate
    reg_all=0.02,      # Regularization
    random_state=42
)

svd_model.fit(trainset)

# Evaluate
predictions = svd_model.test(testset)
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

print(f"\n✅ SVD Model Trained!")
print(f"RMSE: {rmse:.3f}")
print(f"MAE: {mae:.3f}")

In [ ]:
# Make predictions with SVD
user_id = 1
all_items = ratings_df['item_id'].unique()

# Get items user hasn't rated
rated_items = ratings_df[ratings_df['user_id'] == user_id]['item_id'].values
unrated_items = [item for item in all_items if item not in rated_items]

# Predict ratings for unrated items
svd_predictions = []
for item in unrated_items:
    pred = svd_model.predict(user_id, item)
    svd_predictions.append((item, pred.est))

# Sort by predicted rating
svd_predictions.sort(key=lambda x: x[1], reverse=True)

print(f"\n🎯 SVD Recommendations for User {user_id}:")
for item, rating in svd_predictions[:3]:
    print(f"  Item {item}: Predicted Rating = {rating:.2f}")

### Cross-Validation for Model Selection

In [ ]:
# Compare different algorithms
from surprise import SVD, NMF, KNNBasic

algorithms = {
    'SVD': SVD(n_factors=10, random_state=42),
    'NMF': NMF(n_factors=10, random_state=42),
    'KNN-Basic': KNNBasic(k=3)
}

results = {}
for name, algo in algorithms.items():
    cv_results = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=3, verbose=False)
    results[name] = {
        'RMSE': cv_results['test_rmse'].mean(),
        'MAE': cv_results['test_mae'].mean()
    }

# Display results
results_df = pd.DataFrame(results).T
print("\n📊 Algorithm Comparison (3-Fold CV):")
print(results_df.round(3))

# Visualize
results_df.plot(kind='bar', figsize=(10, 5))
plt.title('Recommender Algorithm Performance')
plt.ylabel('Error')
plt.xlabel('Algorithm')
plt.xticks(rotation=45)
plt.legend(title='Metric')
plt.tight_layout()
plt.show()

## Part 4: Content-Based Filtering

### When to Use Content-Based?

**Advantages**:
- ✅ No cold start problem for new items (if you have metadata)
- ✅ Can explain recommendations: "Because this movie is Action/Sci-Fi like Inception"
- ✅ Works with limited user data

**Disadvantages**:
- ❌ Limited serendipity (only recommends similar items)
- ❌ Requires good item features/metadata

**Example**: Recommend movies based on genre, director, actors

In [ ]:
# Create movie dataset with features
movies_data = {
    'movie_id': [101, 102, 103, 104, 105],
    'title': ['Inception', 'Interstellar', 'The Matrix', 'Blade Runner', 'Avatar'],
    'genre': [
        'Action Sci-Fi Thriller',
        'Adventure Drama Sci-Fi',
        'Action Sci-Fi',
        'Sci-Fi Thriller',
        'Action Adventure Sci-Fi'
    ],
    'director': [
        'Christopher Nolan',
        'Christopher Nolan',
        'Wachowskis',
        'Ridley Scott',
        'James Cameron'
    ]
}

movies_df = pd.DataFrame(movies_data)
movies_df['features'] = movies_df['genre'] + ' ' + movies_df['director']

print("Movies Dataset:")
print(movies_df[['title', 'features']])

In [ ]:
# Create TF-IDF vectors from features
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies_df['features'])

print(f"TF-IDF Matrix Shape: {tfidf_matrix.shape}")
print(f"Features: {tfidf.get_feature_names_out()}")

# Calculate movie similarity
movie_similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)
movie_similarity_df = pd.DataFrame(
    movie_similarity,
    index=movies_df['title'],
    columns=movies_df['title']
)

print("\nMovie Similarity Matrix:")
print(movie_similarity_df.round(3))

# Visualize
plt.figure(figsize=(10, 8))
sns.heatmap(movie_similarity_df, annot=True, fmt='.2f', cmap='YlOrRd')
plt.title('Movie Similarity (Content-Based)')
plt.tight_layout()
plt.show()

In [ ]:
# Content-based recommendation function
def recommend_content_based(movie_title, movies_df, similarity_df, top_n=3):
    """
    Recommend movies similar to the given movie.
    """
    # Get similarity scores
    similar_scores = similarity_df[movie_title].sort_values(ascending=False)[1:]  # Exclude self
    
    # Get top N
    top_movies = similar_scores.head(top_n)
    
    # Get movie details
    recommendations = []
    for movie, score in top_movies.items():
        movie_info = movies_df[movies_df['title'] == movie].iloc[0]
        recommendations.append({
            'title': movie,
            'similarity': score,
            'genre': movie_info['genre'],
            'director': movie_info['director']
        })
    
    return recommendations

# Test recommendations
movie = 'Inception'
recommendations = recommend_content_based(movie, movies_df, movie_similarity_df)

print(f"\n🎯 Because you liked '{movie}', you might also like:\n")
for i, rec in enumerate(recommendations, 1):
    print(f"{i}. {rec['title']} (Similarity: {rec['similarity']:.2f})")
    print(f"   Genre: {rec['genre']}")
    print(f"   Director: {rec['director']}\n")

## Part 5: Hybrid Recommender Systems

### Why Hybrid?

**Problem**: Each approach has weaknesses
- Collaborative: Cold start, sparsity
- Content-based: No serendipity, over-specialization

**Solution**: Combine them!

### Hybrid Approaches

1. **Weighted**: Score = α × CF_score + (1-α) × CB_score
2. **Switching**: Use CF if enough data, else CB
3. **Mixed**: Show recommendations from both
4. **Feature Combination**: Use CF scores as features in CB
5. **Cascade**: CF filters, then CB re-ranks
6. **Meta-level**: Use CB to build user profile, then CF

In [ ]:
# Hybrid recommender: Weighted approach
def recommend_hybrid(user_id, item_similarity_df, user_item_matrix, 
                     movie_similarity_df, movies_df, alpha=0.6, top_n=3):
    """
    Hybrid recommender combining collaborative and content-based.
    alpha: weight for collaborative filtering (1-alpha for content-based)
    """
    # Get user's rated items
    user_ratings = user_item_matrix.loc[user_id]
    rated_items = user_ratings[user_ratings > 0]
    unrated_items = user_ratings[user_ratings == 0].index
    
    hybrid_scores = {}
    
    for item in unrated_items:
        # Collaborative filtering score
        cf_score = 0
        cf_denominator = 0
        for rated_item, rating in rated_items.items():
            similarity = item_similarity_df.loc[item, rated_item]
            cf_score += rating * similarity
            cf_denominator += similarity
        cf_score = cf_score / cf_denominator if cf_denominator > 0 else 0
        
        # Content-based score
        cb_score = 0
        cb_count = 0
        item_title = movies_df[movies_df['movie_id'] == item]['title'].iloc[0]
        for rated_item in rated_items.index:
            rated_title = movies_df[movies_df['movie_id'] == rated_item]['title'].iloc[0]
            if rated_title in movie_similarity_df.columns:
                cb_score += movie_similarity_df.loc[item_title, rated_title]
                cb_count += 1
        cb_score = cb_score / cb_count if cb_count > 0 else 0
        
        # Combine scores
        hybrid_scores[item] = alpha * cf_score + (1 - alpha) * cb_score
    
    # Sort and return top N
    sorted_scores = sorted(hybrid_scores.items(), key=lambda x: x[1], reverse=True)
    return sorted_scores[:top_n]

# Test hybrid recommendations
user_id = 1
recommendations = recommend_hybrid(
    user_id, item_similarity_df, user_item_matrix,
    movie_similarity_df, movies_df, alpha=0.6
)

print(f"\n🎯 Hybrid Recommendations for User {user_id}:")
print("(60% Collaborative + 40% Content-Based)\n")
for item, score in recommendations:
    movie_title = movies_df[movies_df['movie_id'] == item]['title'].iloc[0]
    print(f"  {movie_title} (Item {item}): Score = {score:.3f}")

## Part 6: Deep Learning for Recommender Systems

### Neural Collaborative Filtering (NCF)

**Idea**: Replace dot product with neural network
- Traditional CF: rating = user_vector · item_vector
- NCF: rating = Neural_Network(user_vector, item_vector)

**Benefits**:
- Can learn complex, non-linear patterns
- Better than matrix factorization on large datasets
- Used by YouTube, TikTok, Instagram

**Architecture**:
1. User & Item embeddings
2. Concatenate embeddings
3. Deep neural network
4. Output: Predicted rating/probability

In [ ]:
# Install PyTorch
!pip install torch -q

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

print("✅ PyTorch installed!")

In [ ]:
# Neural Collaborative Filtering Model
class NCF(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=16, hidden_layers=[64, 32, 16]):
        super(NCF, self).__init__()
        
        # Embedding layers
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        
        # MLP layers
        layers = []
        input_dim = embedding_dim * 2
        for hidden_dim in hidden_layers:
            layers.append(nn.Linear(input_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.2))
            input_dim = hidden_dim
        
        self.mlp = nn.Sequential(*layers)
        self.output = nn.Linear(hidden_layers[-1], 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, user_ids, item_ids):
        # Get embeddings
        user_embed = self.user_embedding(user_ids)
        item_embed = self.item_embedding(item_ids)
        
        # Concatenate
        x = torch.cat([user_embed, item_embed], dim=1)
        
        # MLP
        x = self.mlp(x)
        x = self.output(x)
        x = self.sigmoid(x) * 5  # Scale to 1-5 rating
        
        return x.squeeze()

print("✅ NCF Model defined!")

In [ ]:
# Prepare data for PyTorch
class RatingsDataset(Dataset):
    def __init__(self, ratings_df):
        # Convert to 0-indexed
        self.users = torch.LongTensor(ratings_df['user_id'].values - 1)
        self.items = torch.LongTensor(ratings_df['item_id'].values - 101)
        self.ratings = torch.FloatTensor(ratings_df['rating'].values)
    
    def __len__(self):
        return len(self.ratings)
    
    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.ratings[idx]

# Create dataset and dataloader
dataset = RatingsDataset(ratings_df)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Initialize model
num_users = ratings_df['user_id'].nunique()
num_items = ratings_df['item_id'].nunique()

model = NCF(num_users, num_items, embedding_dim=16, hidden_layers=[32, 16, 8])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(f"✅ Model initialized!")
print(f"Users: {num_users}, Items: {num_items}")
print(f"\nModel Architecture:")
print(model)

In [ ]:
# Train NCF model
num_epochs = 100
losses = []

model.train()
for epoch in range(num_epochs):
    epoch_loss = 0
    for user_ids, item_ids, ratings in dataloader:
        # Forward pass
        predictions = model(user_ids, item_ids)
        loss = criterion(predictions, ratings)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(dataloader)
    losses.append(avg_loss)
    
    if (epoch + 1) % 20 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

print("\n✅ Training complete!")

# Plot training loss
plt.figure(figsize=(10, 5))
plt.plot(losses)
plt.title('NCF Training Loss')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Make predictions with NCF
model.eval()

user_id = 0  # 0-indexed (User 1)
all_items = torch.LongTensor(range(num_items))
user_ids = torch.LongTensor([user_id] * num_items)

with torch.no_grad():
    predictions = model(user_ids, all_items)

# Get top recommendations
top_items = torch.argsort(predictions, descending=True)

# Filter out already rated items
rated_items = ratings_df[ratings_df['user_id'] == user_id + 1]['item_id'].values - 101

print(f"\n🎯 NCF Recommendations for User {user_id + 1}:\n")
count = 0
for item_idx in top_items:
    if item_idx.item() not in rated_items:
        actual_item_id = item_idx.item() + 101
        pred_rating = predictions[item_idx].item()
        movie_title = movies_df[movies_df['movie_id'] == actual_item_id]['title'].iloc[0]
        print(f"  {movie_title} (Item {actual_item_id}): Predicted Rating = {pred_rating:.2f}")
        count += 1
        if count >= 3:
            break

## Part 7: Evaluation Metrics

### Ranking Metrics

**Precision@K**: Of top K recommendations, how many are relevant?
$$Precision@K = \frac{\text{Relevant items in top K}}{K}$$

**Recall@K**: Of all relevant items, how many are in top K?
$$Recall@K = \frac{\text{Relevant items in top K}}{\text{Total relevant items}}$$

**NDCG@K**: Normalized Discounted Cumulative Gain
- Considers ranking position (top recommendations matter more)
- Industry standard metric

In [ ]:
# Evaluation metrics implementation
def precision_at_k(recommended_items, relevant_items, k):
    """Calculate Precision@K"""
    recommended_k = recommended_items[:k]
    relevant_in_k = len(set(recommended_k) & set(relevant_items))
    return relevant_in_k / k if k > 0 else 0

def recall_at_k(recommended_items, relevant_items, k):
    """Calculate Recall@K"""
    recommended_k = recommended_items[:k]
    relevant_in_k = len(set(recommended_k) & set(relevant_items))
    return relevant_in_k / len(relevant_items) if len(relevant_items) > 0 else 0

def ndcg_at_k(recommended_items, relevant_items, k):
    """Calculate NDCG@K"""
    dcg = 0
    for i, item in enumerate(recommended_items[:k]):
        if item in relevant_items:
            # Discounted gain (1/log2(position+2))
            dcg += 1 / np.log2(i + 2)
    
    # Ideal DCG (best possible ranking)
    idcg = sum([1 / np.log2(i + 2) for i in range(min(len(relevant_items), k))])
    
    return dcg / idcg if idcg > 0 else 0

# Example evaluation
recommended = [102, 104, 105, 106, 107]  # Recommended items
relevant = [102, 104, 108, 109]  # Actually relevant items

k = 3
print(f"Recommended (top {k}): {recommended[:k]}")
print(f"Relevant items: {relevant}")
print(f"\nMetrics @ K={k}:")
print(f"  Precision@{k}: {precision_at_k(recommended, relevant, k):.3f}")
print(f"  Recall@{k}: {recall_at_k(recommended, relevant, k):.3f}")
print(f"  NDCG@{k}: {ndcg_at_k(recommended, relevant, k):.3f}")

## 🏆 CAPSTONE PROJECT: E-Commerce Recommendation Engine

### Project Goal
Build a complete recommendation system for an e-commerce platform using real-world data.

### Requirements
1. Use collaborative filtering (SVD)
2. Implement content-based filtering
3. Create hybrid recommender
4. Evaluate with Precision@K, Recall@K, NDCG
5. Handle cold start for new users/items
6. Bonus: Add deep learning model (NCF)

In [ ]:
# Load real e-commerce dataset
# We'll create a synthetic but realistic dataset
np.random.seed(42)

# Generate data
n_users = 100
n_products = 50
n_ratings = 500

# Create ratings
user_ids = np.random.randint(1, n_users + 1, n_ratings)
product_ids = np.random.randint(1, n_products + 1, n_ratings)
ratings = np.random.randint(1, 6, n_ratings)

ecommerce_ratings = pd.DataFrame({
    'user_id': user_ids,
    'product_id': product_ids,
    'rating': ratings
})

# Remove duplicates (user can't rate same product twice)
ecommerce_ratings = ecommerce_ratings.drop_duplicates(subset=['user_id', 'product_id'])

print(f"✅ E-Commerce Dataset Created!")
print(f"Users: {ecommerce_ratings['user_id'].nunique()}")
print(f"Products: {ecommerce_ratings['product_id'].nunique()}")
print(f"Ratings: {len(ecommerce_ratings)}")
print(f"Sparsity: {(1 - len(ecommerce_ratings) / (n_users * n_products)) * 100:.2f}%")

print("\nSample data:")
print(ecommerce_ratings.head())

In [ ]:
# Product metadata
categories = ['Electronics', 'Clothing', 'Home', 'Books', 'Toys', 'Sports']
brands = ['BrandA', 'BrandB', 'BrandC', 'BrandD', 'BrandE']

products_data = []
for i in range(1, n_products + 1):
    products_data.append({
        'product_id': i,
        'name': f'Product_{i}',
        'category': np.random.choice(categories),
        'brand': np.random.choice(brands),
        'price': np.random.uniform(10, 500)
    })

products_df = pd.DataFrame(products_data)
products_df['features'] = products_df['category'] + ' ' + products_df['brand']

print("Products Dataset:")
print(products_df.head(10))

In [ ]:
# Step 1: Collaborative Filtering with SVD
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ecommerce_ratings[['user_id', 'product_id', 'rating']], reader)

trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Train SVD
svd = SVD(n_factors=20, n_epochs=30, lr_all=0.005, reg_all=0.02, random_state=42)
svd.fit(trainset)

# Evaluate
predictions = svd.test(testset)
rmse = accuracy.rmse(predictions, verbose=False)
mae = accuracy.mae(predictions, verbose=False)

print(f"✅ Collaborative Filtering Model Trained!")
print(f"RMSE: {rmse:.3f}")
print(f"MAE: {mae:.3f}")

In [ ]:
# Step 2: Content-Based Filtering
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# TF-IDF on product features
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(products_df['features'])
content_similarity = cosine_similarity(tfidf_matrix)

print(f"✅ Content-Based Model Created!")
print(f"Similarity Matrix Shape: {content_similarity.shape}")

In [ ]:
# Step 3: Hybrid Recommender System
class HybridRecommender:
    def __init__(self, svd_model, content_sim, products_df, ratings_df, alpha=0.7):
        self.svd = svd_model
        self.content_sim = content_sim
        self.products_df = products_df
        self.ratings_df = ratings_df
        self.alpha = alpha  # Weight for collaborative filtering
    
    def recommend(self, user_id, top_n=5):
        # Get all products
        all_products = self.products_df['product_id'].values
        
        # Get products user already rated
        rated_products = self.ratings_df[self.ratings_df['user_id'] == user_id]['product_id'].values
        unrated_products = [p for p in all_products if p not in rated_products]
        
        scores = {}
        for product in unrated_products:
            # Collaborative filtering score
            cf_pred = self.svd.predict(user_id, product)
            cf_score = cf_pred.est / 5.0  # Normalize to 0-1
            
            # Content-based score
            cb_score = 0
            if len(rated_products) > 0:
                # Average similarity to user's rated products
                product_idx = product - 1
                similarities = []
                for rated_prod in rated_products:
                    rated_idx = rated_prod - 1
                    similarities.append(self.content_sim[product_idx, rated_idx])
                cb_score = np.mean(similarities)
            
            # Hybrid score
            scores[product] = self.alpha * cf_score + (1 - self.alpha) * cb_score
        
        # Sort and return top N
        sorted_products = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        return sorted_products[:top_n]
    
    def explain(self, user_id, product_id):
        """Explain why a product is recommended"""
        # Get user's highly rated products
        user_ratings = self.ratings_df[
            (self.ratings_df['user_id'] == user_id) & 
            (self.ratings_df['rating'] >= 4)
        ]
        
        if len(user_ratings) == 0:
            return "Based on your general preferences (cold start)"
        
        # Find most similar product user liked
        product_idx = product_id - 1
        max_sim = 0
        most_similar = None
        
        for _, row in user_ratings.iterrows():
            rated_prod = row['product_id']
            rated_idx = rated_prod - 1
            sim = self.content_sim[product_idx, rated_idx]
            if sim > max_sim:
                max_sim = sim
                most_similar = rated_prod
        
        if most_similar:
            similar_prod = self.products_df[self.products_df['product_id'] == most_similar].iloc[0]
            rec_prod = self.products_df[self.products_df['product_id'] == product_id].iloc[0]
            return f"Because you liked {similar_prod['name']} ({similar_prod['category']}), you might like this {rec_prod['category']} product!"
        
        return "Based on what similar users enjoyed"

# Create hybrid recommender
hybrid_rec = HybridRecommender(svd, content_similarity, products_df, ecommerce_ratings, alpha=0.7)

print("✅ Hybrid Recommender System Created!")
print("Alpha = 0.7 (70% collaborative, 30% content-based)")

In [ ]:
# Step 4: Generate Recommendations
test_user = 5
recommendations = hybrid_rec.recommend(test_user, top_n=5)

print(f"\n🎯 Top 5 Recommendations for User {test_user}:\n")
print("="*70)

for i, (product_id, score) in enumerate(recommendations, 1):
    product = products_df[products_df['product_id'] == product_id].iloc[0]
    explanation = hybrid_rec.explain(test_user, product_id)
    
    print(f"{i}. {product['name']}")
    print(f"   Category: {product['category']} | Brand: {product['brand']} | Price: ${product['price']:.2f}")
    print(f"   Score: {score:.3f}")
    print(f"   Why: {explanation}")
    print()

print("="*70)

In [ ]:
# Step 5: Evaluation
def evaluate_recommender(recommender, ratings_df, n_users=20, k=5):
    """Evaluate recommender system performance"""
    precisions = []
    recalls = []
    ndcgs = []
    
    # Sample users who have enough ratings
    user_counts = ratings_df['user_id'].value_counts()
    test_users = user_counts[user_counts >= 5].index[:n_users]
    
    for user_id in test_users:
        # Get user's highly rated products (4-5 stars) as ground truth
        relevant_products = ratings_df[
            (ratings_df['user_id'] == user_id) & 
            (ratings_df['rating'] >= 4)
        ]['product_id'].values
        
        if len(relevant_products) == 0:
            continue
        
        # Get recommendations
        recommendations = recommender.recommend(user_id, top_n=k)
        recommended_products = [prod_id for prod_id, _ in recommendations]
        
        # Calculate metrics
        precisions.append(precision_at_k(recommended_products, relevant_products, k))
        recalls.append(recall_at_k(recommended_products, relevant_products, k))
        ndcgs.append(ndcg_at_k(recommended_products, relevant_products, k))
    
    return {
        'precision': np.mean(precisions),
        'recall': np.mean(recalls),
        'ndcg': np.mean(ndcgs)
    }

# Evaluate
metrics = evaluate_recommender(hybrid_rec, ecommerce_ratings, n_users=30, k=5)

print("\n📊 Hybrid Recommender Performance (K=5):\n")
print(f"Precision@5: {metrics['precision']:.3f}")
print(f"Recall@5: {metrics['recall']:.3f}")
print(f"NDCG@5: {metrics['ndcg']:.3f}")

# Visualize
plt.figure(figsize=(8, 5))
metrics_list = ['Precision@5', 'Recall@5', 'NDCG@5']
values = [metrics['precision'], metrics['recall'], metrics['ndcg']]

bars = plt.bar(metrics_list, values, color=['#2ecc71', '#3498db', '#e74c3c'])
plt.ylabel('Score')
plt.title('Hybrid Recommender Performance')
plt.ylim(0, 1)

# Add value labels on bars
for bar, value in zip(bars, values):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
             f'{value:.3f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

## 🎓 Key Takeaways

### What You've Learned

1. **Collaborative Filtering**
   - User-based and item-based approaches
   - When to use each
   
2. **Matrix Factorization**
   - SVD for sparse matrices
   - The Netflix Prize technique
   - Production-ready with Surprise library
   
3. **Content-Based Filtering**
   - TF-IDF for feature extraction
   - Cosine similarity for recommendations
   - Explainable recommendations
   
4. **Hybrid Systems**
   - Combining CF and CB
   - Handling cold start
   - Best of both worlds
   
5. **Deep Learning**
   - Neural Collaborative Filtering
   - Embeddings + MLP
   - State-of-the-art performance
   
6. **Evaluation**
   - Precision, Recall, NDCG
   - Ranking metrics
   - Real-world evaluation

### Interview-Ready Skills ✅

**You can now answer:**
- "Explain collaborative filtering vs content-based filtering"
- "How do you handle cold start problem?"
- "What is matrix factorization? Why is it useful?"
- "How do you evaluate a recommender system?"
- "Describe a hybrid recommender system you've built"

**You can now build:**
- E-commerce product recommendations (Amazon-style)
- Content recommendations (Netflix-style)
- Social media feeds (TikTok-style)
- Ad targeting systems

### Real-World Applications

**E-Commerce** (Woolworths, Amazon):
- Product recommendations
- "Frequently bought together"
- Personalized search results

**Property/Real Estate** (REA Group):
- Property recommendations based on user search history
- Similar properties suggestion
- Location-based recommendations

**Streaming** (Netflix, Spotify):
- Content recommendations
- Personalized playlists/homepages
- Continue watching suggestions

**Social Media** (TikTok, Instagram):
- Feed ranking
- Content discovery
- Friend suggestions

---

## 🚀 Next Steps

1. **Add to your portfolio**: Deploy this e-commerce recommender as a web app
2. **Learn more**: Learning to Rank (LambdaMART), Multi-Armed Bandits
3. **Practice**: Kaggle competitions (MovieLens, Amazon reviews)
4. **Scale**: Learn about distributed recommenders (Spark MLlib)

---

## 📚 Additional Resources

**Papers**:
- Netflix Prize winning solution
- Neural Collaborative Filtering (He et al., 2017)
- Deep Learning based Recommender System (Zhang et al., 2019)

**Libraries**:
- Surprise: For collaborative filtering
- LightFM: Hybrid recommender
- Implicit: Fast ALS implementation
- RecBole: Comprehensive RecSys library

**Datasets**:
- MovieLens: Movie ratings
- Amazon Reviews: Product ratings
- LastFM: Music listening history
- Yelp: Restaurant reviews

---

## ✅ Job Market Relevance

**REA Group Job Posting**: ✅
> "Experience with recommender systems (Collaborative Filtering, Content-Based, Hybrid)"

**Woolworths ML Engineer**: ✅
> "Build personalized customer experiences"

**Industry Usage**: ✅
- 35% of Amazon revenue from recommendations
- 75% of Netflix viewing from recommendations
- 60% of YouTube watch time from recommendations

**You are now job-ready for RecSys roles!** 🎉

---

**Congratulations!** You've completed Day 1 of Week 22. Tomorrow: Time Series Forecasting! 🚀